# optuna_optimiser_g_n_d_multiplier

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from bp import data_gather_from_files,run_strategy_optimised,run_strategy_eval
import seaborn as sns
import time
import scipy
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def error_check(params,n):
    if len(params) != n:
        raise IndexError('The number of parameters is not correct')

def generate_date_ranges_for_walk_forward(start_month_year, end_month_year, day=15, n_months = 1):
    # Initialize an empty list to store the date ranges
    date_ranges = []
    if day not in range(1, 29):
        raise ValueError('Day must be between 1 and 28')
    # Convert the input strings to datetime objects, using the given day
    start_date = datetime.strptime(f"{day} {start_month_year}", '%d %b %Y')
    end_date = datetime.strptime(f"{day} {end_month_year}", '%d %b %Y')
    
    # Generate the date ranges
    current_date = start_date
    while current_date <= end_date:
        next_date = current_date + relativedelta(months=n_months)
        date_range = [current_date.strftime('%d %b %Y'), (next_date - relativedelta(days=1)).strftime('%d %b %Y')]
        date_ranges.append(date_range)
        current_date = next_date
    
    return date_ranges[:-1]

def get_previous_n_months(end_date_str, n_months):
    # Convert the input string to a datetime object
    end_date = datetime.strptime(end_date_str, '%d %b %Y')
    
    # Calculate the start date
    start_date = end_date - relativedelta(months=n_months)
    
    # Create the date range
    date_range = [start_date.strftime('%d %b %Y'), (end_date - relativedelta(days=1)).strftime('%d %b %Y')]
    
    return date_range

In [3]:
def walk_forward_analysis(evaluation_start, evaluation_end, evaluation_day,parameters,optimization_function = None, optimizer_params =[],  lookback_in_months = 6,evaluation_period = 3):
    generated_date_ranges = generate_date_ranges_for_walk_forward(evaluation_start, evaluation_end,evaluation_day,n_months = evaluation_period)
    df = {}
    for dates in generated_date_ranges:
        train_period = get_previous_n_months(dates[0], lookback_in_months)
        train_data = data_gather_from_files(train_period[0],train_period[1])['EURUSD.mid']
        print('Data gathered for training period: ',train_period[0],train_period[1])
        test_data = data_gather_from_files(dates[0],dates[1])['EURUSD.mid']
        print('Data gathered for testing period: ',dates[0],dates[1])
        max_loss, U_PNL, max_position, R_PNL,profit,optimal_params = optimization_function(train_data,test_data,parameters,optimizer_params)
        print('Optimal parameters are: ',optimal_params)
        print('Max loss, U_PNL, max_position, R_PNL,profit are: ',max_loss, U_PNL, max_position, R_PNL,profit)
        df[dates[0] +'-'+ dates[1]] = [max_loss, U_PNL, max_position, R_PNL,profit]
    df = pd.DataFrame(df).T
    df.columns = ['max_loss', 'min_U_PNL', 'max_position', 'R_PNL','profit']
    return df

## Pyswarm Group 2

In [15]:
from pyswarm import pso

def pyswarm_optimiser_g_n_d_multiplier(train_data,test_data,parameters,optimization_params):
    n_trials = optimization_params[0] 

    error_check(parameters,5)
        
    grid_params = parameters[0]
    position_params = parameters[1]
    depth_params = parameters[2]
    multiplier_params = parameters[3]
    lookback_params = parameters[4]

    # PSO Bounds
    lb = [
        grid_params[0]/grid_params[2],
        position_params[0]/position_params[2],
        depth_params[0],
        multiplier_params[0]/ multiplier_params[2],
        0  
    ]
    ub = [
        grid_params[1]/grid_params[2],
        position_params[1]/position_params[2],
        depth_params[1],
        multiplier_params[1]/ multiplier_params[2],
        len(lookback_params) - 1
    ]
    def objective_pso(params, tick_data):

        G = int(params[0])* grid_params[2]
        n = int(params[1]) * position_params[2]
        d = int(params[2])
        multiplier = int(params[3]) *  multiplier_params[2]
        lookback_index = int(params[4])
        lookback = lookback_params[lookback_index]  
        
        max_loss,U_PNL,max_position,R_PNL,profit = run_strategy_optimised(tick_data, G, n, d, multiplier = multiplier,lookback = lookback)
        
        # Constraints
        if max_position > 10e6:
            return float('inf')
        if U_PNL < -150e3:
            return float('inf')
        if max_loss < -500e3:
            return float('inf')
        # just some bounds on the multiplier dont have to use this condition.
        if (multiplier**d)*n > 10e6:
            return float('inf')
        return -profit # We negate the value since we want to maximize Profit

    params_opt, _ = pso(lambda params: objective_pso(params,train_data), lb, ub, swarmsize=30, maxiter=n_trials)  

    optimal_g = int(params_opt[0])* grid_params[2]
    optimal_n = int(params_opt[1]) * position_params[2]
    optimal_d = int(params_opt[2])
    optimal_m = int(params_opt[3]) *  multiplier_params[2]
    optimal_l_index = int(params_opt[4])
    optimal_l = lookback_params[optimal_l_index]
    
    print("optimisation completed")
    max_loss, U_PNL, max_position, R_PNL,profit = run_strategy_optimised(test_data, optimal_g,optimal_n,optimal_d,multiplier = optimal_m,lookback = optimal_l)
    return max_loss, U_PNL, max_position, R_PNL,profit,[optimal_g,optimal_n,optimal_d,optimal_m,optimal_l]


In [19]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
depth_params = [3,12,1]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_depth_params = ((depth_params[1]-depth_params[0])/depth_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)
print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of depth params:-',(n_depth_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_depth_params*n_multiplier_params*n_lookback_params))

n_trials = 100
optimizer_param = [n_trials]

print('search_space_explored:-',(n_trials/(n_grid_params*n_lot_params*n_depth_params*n_multiplier_params*n_lookback_params))*100,'%')
parameters1 = [grid_params,lot_params,depth_params,multiplier_params,lookback_params]
results1 = walk_forward_analysis('jan 2021','jan 2022',1,parameters1,optimization_function=pyswarm_optimiser_g_n_d_multiplier,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)
c

number of grid params:- 18.0
number of lot params:- 19.0
number of depth params:- 9.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 430920.0
search_space_explored:- 0.02320616355704075 %


Data gathered for training period:  01 Jul 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Mar 2021


Exception: No improvement for 10 iterations, stopping optimization.

# CUT

In [5]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
depth_params = [3,12,1]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_depth_params = ((depth_params[1]-depth_params[0])/depth_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)
print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of depth params:-',(n_depth_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_depth_params*n_multiplier_params*n_lookback_params))

n_trials = 500
optimizer_param = [n_trials]

print('search_space_explored:-',(n_trials/(n_grid_params*n_lot_params*n_depth_params*n_multiplier_params*n_lookback_params))*100,'%')
parameters1 = [grid_params,lot_params,depth_params,multiplier_params,lookback_params]
results1 = walk_forward_analysis('jan 2021','jan 2022',1,parameters1,optimization_function=pyswarm_optimiser_g_n_d_multiplier,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)
c

number of grid params:- 18.0
number of lot params:- 19.0
number of depth params:- 9.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 430920.0
search_space_explored:- 0.11603081778520374 %
Data gathered for training period:  01 Jul 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Mar 2021
Stopping search: maximum iterations reached --> 100
optimisation completed
Optimal parameters are:  [0.0045000000000000005, 300000, 4, 1.9000000000000001, 30]
Max loss, U_PNL, max_position, R_PNL,profit are:  -31050.0 -42150.0 4406400.0 33268.0 108540.0
Data gathered for training period:  01 Oct 2020 31 Mar 2021
Data gathered for testing period:  01 Apr 2021 30 Jun 2021
Stopping search: maximum iterations reached --> 100
optimisation completed
Optimal parameters are:  [0.0085, 1300000, 8, 1.2000000000000002, 9]
Max loss, U_PNL, max_position, R_PNL,profit are:  -255800.0 -110500.0 10829000.0 -255800.0 -255800.0
Data gathered for training

,max_loss,min_U_PNL,max_position,R_PNL,profit
01 Jan 2021-31 Mar 2021,-31050.0,-42150.0,4406400.0,33268.0,108540.0
01 Apr 2021-30 Jun 2021,-255800.0,-110500.0,10829000.0,-255800.0,-255800.0
01 Jul 2021-30 Sep 2021,0.0,-20000.0,4760000.0,100800.0,100800.0
01 Oct 2021-31 Dec 2021,-63000.0,-75600.0,6300000.0,42840.0,42840.0


In [5]:
grid_params = [0.001,0.01,0.0005]
lot_params = [100000,2000000,100000]
depth_params = [3,12,1]
multiplier_params = [1,3,0.1]
lookback_params = [9,20,30,50,100,150,200]

n_grid_params = ((grid_params[1]-grid_params[0])/grid_params[2])
n_lot_params = ((lot_params[1]-lot_params[0])/lot_params[2])
n_depth_params = ((depth_params[1]-depth_params[0])/depth_params[2])
n_multiplier_params = ((multiplier_params[1]-multiplier_params[0])/multiplier_params[2])
n_lookback_params = len(lookback_params)
print('number of grid params:-',(n_grid_params))
print('number of lot params:-',(n_lot_params))
print('number of depth params:-',(n_depth_params))
print('number of multiplier params:-',(n_multiplier_params))
print('number of lookback params:-',(n_lookback_params))
print('total_number_of_combinations:-',(n_grid_params*n_lot_params*n_depth_params*n_multiplier_params*n_lookback_params))

n_trials = 500
optimizer_param = [n_trials]

print('search_space_explored:-',(n_trials/(n_grid_params*n_lot_params*n_depth_params*n_multiplier_params*n_lookback_params))*100,'%')
parameters1 = [grid_params,lot_params,depth_params,multiplier_params,lookback_params]
results1 = walk_forward_analysis('jan 2021','jan 2022',1,parameters1,optimization_function=pyswarm_optimiser_g_n_d_multiplier,optimizer_params=optimizer_param,lookback_in_months=6,evaluation_period=3)
results1

number of grid params:- 18.0
number of lot params:- 19.0
number of depth params:- 9.0
number of multiplier params:- 20.0
number of lookback params:- 7
total_number_of_combinations:- 430920.0
search_space_explored:- 0.11603081778520374 %
Data gathered for training period:  01 Jul 2020 31 Dec 2020
Data gathered for testing period:  01 Jan 2021 31 Mar 2021
Stopping search: maximum iterations reached --> 500
optimisation completed
Optimal parameters are:  [0.0045000000000000005, 200000, 8, 1.2000000000000002, 200]
Max loss, U_PNL, max_position, R_PNL,profit are:  -10800.0 -21728.5714 2448000.0 30855.2383 54400.0
Data gathered for training period:  01 Oct 2020 31 Mar 2021
Data gathered for testing period:  01 Apr 2021 30 Jun 2021
Stopping search: maximum iterations reached --> 500
optimisation completed
Optimal parameters are:  [0.0085, 1100000, 9, 1.2000000000000002, 9]
Max loss, U_PNL, max_position, R_PNL,profit are:  -168300.0 -140250.0 8078400.0 -65450.0 -65450.0
Data gathered for train

,max_loss,min_U_PNL,max_position,R_PNL,profit
01 Jan 2021-31 Mar 2021,-10800.0,-21728.5714,2448000.0,30855.2383,54400.0
01 Apr 2021-30 Jun 2021,-168300.0,-140250.0000,8078400.0,-65450.0000,-65450.0
01 Jul 2021-30 Sep 2021,0.0,-20000.0000,4760000.0,100800.0000,100800.0
01 Oct 2021-31 Dec 2021,-72000.0,-86400.0000,7200000.0,46080.0000,48960.0
